In [32]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import *

In [49]:
url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/'+\
'master/dataset/HAR_dataset/features.txt'

feature_name = pd.read_csv(url, sep= '\s+', header=None,
                           names=['column_index' , 'column_name'])

print(f'{len(feature_name)} columns in the dataset')
# print('-------------------\n')
# print(feature_name.head(3))
# print('-------------------\n')

feature_name = feature_name.iloc[:, 1].tolist()

X_train_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/'+\
'master/dataset/HAR_dataset/train/X_train.txt'
X_test_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/' +\
'master/dataset/HAR_dataset/test/X_test.txt'

X_train = pd.read_csv(X_train_url, sep='\s+' , header=None)
X_test = pd.read_csv(X_test_url, sep='\s+' , header=None)

X_train.columns = feature_name
X_test.columns = feature_name

y_train_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/' +\
'master/dataset/HAR_dataset/train/y_train.txt'
y_test_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/' +\
'master/dataset/HAR_dataset/test/y_test.txt'

y_train = pd.read_csv(y_train_url, sep='\s+', header=None, names=['action'])
y_test = pd.read_csv(y_test_url, sep='\s+', header=None, names=['action'])


561 columns in the dataset


### Standard Decision Tree (with multi depth)

In [80]:
from sklearn.model_selection import GridSearchCV

dt_clf = DecisionTreeClassifier(random_state=1)


depth_range = np.arange(2,52,2).tolist()
params = {'max_depth' : depth_range}

cross_validation_fold = 5

grid_cv = GridSearchCV(dt_clf, param_grid= params, scoring='accuracy', 
                       cv=cross_validation_fold, return_train_score=True)

print(f'Decision Tree depth test : depth {depth_range[0]} to {depth_range[-1]}')
print(f'for each depth, {cross_validation_fold} fold for validation')

Decision Tree depth test : depth 2 to 50


In [81]:
grid_cv.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=1),
             param_grid={'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22,
                                       24, 26, 28, 30, 32, 34, 36, 38, 40, 42,
                                       44, 46, 48, 50]},
             return_train_score=True, scoring='accuracy')

In [91]:
result = pd.DataFrame(grid_cv.cv_results_)
result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.518946,0.009978,0.004737,0.000306,2,{'max_depth': 2},0.545207,0.543848,0.545578,0.544898,0.544218,0.544750,0.000635,25,0.544975,0.544975,0.544883,0.545053,0.545223,0.545022,0.000114
1,0.944042,0.003949,0.004706,0.000246,4,{'max_depth': 4},0.762746,0.874235,0.827211,0.848980,0.878912,0.838417,0.042157,8,0.906478,0.901037,0.892214,0.894084,0.903434,0.899449,0.005458
2,1.356859,0.009227,0.004615,0.000269,6,{'max_depth': 6},0.815092,0.870156,0.799320,0.862585,0.870068,0.843444,0.030132,3,0.963272,0.939126,0.946107,0.931826,0.944237,0.944913,0.010428
3,1.717059,0.017025,0.004595,0.000138,8,{'max_depth': 8},0.819850,0.824609,0.838776,0.877551,0.902041,0.852565,0.031982,1,0.985887,0.985377,0.984529,0.977559,0.980109,0.982692,0.003281
4,2.007069,0.058039,0.004619,0.000397,10,{'max_depth': 10},0.787899,0.815092,0.842857,0.886395,0.886395,0.843727,0.038932,2,0.992348,0.995919,0.994560,0.992690,0.991499,0.993403,0.001607


### Random Forest 

In [95]:
from sklearn.ensemble import RandomForestClassifier

depth_range = np.arange(2,10,2).tolist()

params = {
    'max_depth' : depth_range,
    'n_estimators' : [50 ,100 ,200], # number of trees 단순히 많은 경우의 수를 돌리기 위해서 나눈것
    'min_samples_leaf' : [8, 12],
    'min_samples_split' : [8, 12]
}

rf_clf = RandomForestClassifier(random_state=1)
grid_cv = GridSearchCV(rf_clf, param_grid= params, cv=5)
grid_cv.fit(X_train, y_train)

/home/sunnypc/dev/ML/.venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/sunnypc/dev/ML/.venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/sunnypc/dev/ML/.venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/sunnypc/dev/ML/.venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
             param_grid={'max_depth': [2, 4, 6, 8], 'min_samples_leaf': [8, 12],
                         'min_samples_split': [8, 12],
                         'n_estimators': [50, 100, 200]})

In [100]:
random_forest_result = pd.DataFrame(grid_cv.cv_results_).sort_values('rank_test_score')
random_forest_result[['rank_test_score' , 'mean_test_score' , 'param_n_estimators', 'param_max_depth']]

,rank_test_score,mean_test_score,param_n_estimators,param_max_depth
36,1,0.922339,50,8
39,1,0.922339,50,8
41,3,0.921931,200,8
38,3,0.921931,200,8
47,5,0.921250,200,8
44,5,0.921250,200,8
37,7,0.920435,100,8
40,7,0.920435,100,8
43,9,0.918937,100,8
46,9,0.918937,100,8


In [105]:
import manim

ModuleNotFoundError: No module named 'manim'